<a href="https://colab.research.google.com/github/renya1993/Learning/blob/main/first_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optimizers

In [13]:
class MLP(nn.Module):
  def __init__(self,input_dim,hidden_dim,output_dim):
    super().__init__()
    self.l1 = nn.Linear(input_dim,hidden_dim)
    self.a1 = nn.Sigmoid()
    self.l2 = nn.Linear(hidden_dim,output_dim)
    self.a2 = nn.Sigmoid()

    self.layers = [self.l1,self.a1,self.l2,self.a2]

  def forward(self,x):
    for layer in self.layers:
      x = layer(x)
    return x


In [14]:
np.random.seed(123)
torch.manual_seed(123)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MLP(2,3,1).to(device)

In [22]:
N=300
x,t = datasets.make_moons(N,noise=0.3)
t = t.reshape(N,1)

x_train,x_test,t_train,t_test = train_test_split(x,t,test_size=0.2)



In [25]:
## 2クラスの誤差はこれで求める　中身は、E=1/N*tlogy +(1-t)*log(1-y)
criterion = nn.BCELoss()

## model.parameters()が更新したいパラメータ、t,x
optimizer = optimizers.SGD(model.parameters(), lr=0.1)

def compute_loss(t,y):
  return criterion(y,t)

def train_step(x,t):
  model.train() #訓練モード
  preds = model(x) 
  loss = compute_loss(t,preds) #これが最小になった時、いいw,bが手に入る。
  optimizer.zero_grad() #勾配の初期化 :pytorchでは、勾配が勝手に足されていくので、それを避けるため。前回の分が計算に入っちゃう。
  loss.backward() #損失関数E=1/N*tlogy +(1-t)*log(1-y)のx,tについて微分を計算。
  optimizer.step() #w←de/dwの計算
  return loss

epochs = 300
batch_size = 10
n_batches = x_train.shape[0]//batch_size

"""
学習
"""

for epoch in range(epochs):
  print("------------",epoch+1,"---------------")
  train_loss = 0
  x_,t_ = shuffle(x_train,t_train)
  x_ = torch.Tensor(x_).to(device)
  t_ = torch.Tensor(t_).to(device)

  for n_batch in range(n_batches):
    start = n_batch*batch_size
    end = start + batch_size
    loss = train_step(x_[start:end],t_[start:end])
    train_loss += loss.item() #損失関数は、Σ全データなので。こう言う処理をする。フルスクラッチの場合.sum(axis=1)とかで対処したりする。

  print('epoch: {}, loss: {:.3}'.format(
      epoch+1,
      train_loss
  ))
"""
モデルの評価
"""

def test_step(x,t):
  x = torch.Tensor(x).to(device)
  t = torch.Tensor(t).to(device)
  model.eval()
  preds = model(x)
  loss = compute_loss(t,preds)
  return loss,preds

loss,preds = test_step(x_test,t_test)
test_loss = loss.item()
preds = preds.data.cpu().numpy() >0.5 #0.5より大きい時は、True=1、0.5より小さければfalse=0

"""
preds :テンソル型
preds.data 勾配情報を含まないテンソル型
preds.data.cpu() CPUのテンソル形
preds.data.cpu().numpy()  Numpy配列
"""

test_acc = accuracy_score(t_test,preds)
print(test_loss,test_acc)

epoch: 1, loss: 5.96
epoch: 2, loss: 5.95
epoch: 3, loss: 5.93
epoch: 4, loss: 5.95
epoch: 5, loss: 5.95
epoch: 6, loss: 5.95
epoch: 7, loss: 5.94
epoch: 8, loss: 5.94
epoch: 9, loss: 5.95
epoch: 10, loss: 5.96
epoch: 11, loss: 5.94
epoch: 12, loss: 5.96
epoch: 13, loss: 5.96
epoch: 14, loss: 5.96
epoch: 15, loss: 5.93
epoch: 16, loss: 5.95
epoch: 17, loss: 5.96
epoch: 18, loss: 5.95
epoch: 19, loss: 5.94
epoch: 20, loss: 5.94
epoch: 21, loss: 5.95
epoch: 22, loss: 5.95
epoch: 23, loss: 5.93
epoch: 24, loss: 5.94
epoch: 25, loss: 5.94
epoch: 26, loss: 5.93
epoch: 27, loss: 5.97
epoch: 28, loss: 5.93
epoch: 29, loss: 5.94
epoch: 30, loss: 5.94
epoch: 31, loss: 5.94
epoch: 32, loss: 5.94
epoch: 33, loss: 5.92
epoch: 34, loss: 5.92
epoch: 35, loss: 5.94
epoch: 36, loss: 5.94
epoch: 37, loss: 5.94
epoch: 38, loss: 5.91
epoch: 39, loss: 5.93
epoch: 40, loss: 5.94
epoch: 41, loss: 5.94
epoch: 42, loss: 5.94
epoch: 43, loss: 5.93
epoch: 44, loss: 5.97
epoch: 45, loss: 5.94
epoch: 46, loss: 5.

In [4]:
"""
y_trueが最小値を取る値を調べる
"""

x = torch.tensor(3.0, requires_grad = True)
y_true = 3*x**2+7
op = torch.optim.SGD([x], lr=0.1)


epochs = 10
for i in range(epochs):
  op.zero_grad()
  y_true.backward() #これが勝手に、y=6*x(y_trueの微分した値)を作ってくれている。
  op.step()
  print("微分した値",x) #←この値が最小の時、y2は０に近く。最小値となる。
  y_true = 3*x**2+7
  print("求めたい値:",y_true)



微分した値 tensor(1.2000, requires_grad=True)
求めたい値: tensor(11.3200, grad_fn=<AddBackward0>)
微分した値 tensor(0.4800, requires_grad=True)
求めたい値: tensor(7.6912, grad_fn=<AddBackward0>)
微分した値 tensor(0.1920, requires_grad=True)
求めたい値: tensor(7.1106, grad_fn=<AddBackward0>)
微分した値 tensor(0.0768, requires_grad=True)
求めたい値: tensor(7.0177, grad_fn=<AddBackward0>)
微分した値 tensor(0.0307, requires_grad=True)
求めたい値: tensor(7.0028, grad_fn=<AddBackward0>)
微分した値 tensor(0.0123, requires_grad=True)
求めたい値: tensor(7.0005, grad_fn=<AddBackward0>)
微分した値 tensor(0.0049, requires_grad=True)
求めたい値: tensor(7.0001, grad_fn=<AddBackward0>)
微分した値 tensor(0.0020, requires_grad=True)
求めたい値: tensor(7.0000, grad_fn=<AddBackward0>)
微分した値 tensor(0.0008, requires_grad=True)
求めたい値: tensor(7.0000, grad_fn=<AddBackward0>)
微分した値 tensor(0.0003, requires_grad=True)
求めたい値: tensor(7.0000, grad_fn=<AddBackward0>)


In [ ]:
 loss = compute_loss(t,preds) #これが最小になった時、いいw,bが手に入る。
  optimizer.zero_grad() #勾配の初期化 :pytorchでは、勾配が勝手に足されていくので、それを避けるため。前回の分が計算に入っちゃう。
  loss.backward() #損失関数E=1/N*tlogy +(1-t)*log(1-y)のx,tについて微分を計算。
  optimizer.step() #w←de/dwの計算
  return loss